In [16]:
import pandas as pd
import numpy as np

data1 = pd.read_csv('../database/pseudo_observations/vas.txt')
data1 = np.array(data1.iloc[::2, :])
data1 = np.array([float(i[0]) for i in data1])

data2 = pd.read_csv('../database/pseudo_observations/LQD.txt')
data2 = np.array(data2.iloc[::2, :])
data2 = np.array([float(i[0]) for i in data2])

data3 = pd.read_csv('../database/pseudo_observations/HYG.txt')
data3 = np.array(data3.iloc[::2, :])
data3 = np.array([float(i[0]) for i in data3])

data4 = pd.read_csv('../database/pseudo_observations/JPY.txt')
data4 = np.array(data4.iloc[::2, :])
data4 = np.array([float(i[0]) for i in data4])

data5 = pd.read_csv('../database/pseudo_observations/IVV.txt')
data5 = np.array(data5.iloc[::2, :])
data5 = np.array([float(i[0]) for i in data5])

data6 = pd.read_csv('../database/pseudo_observations/GLD.txt')
data6 = np.array(data6.iloc[::2, :])
data6 = np.array([float(i[0]) for i in data6])

data7 = pd.read_csv('../database/pseudo_observations/1321.txt')
data7 = np.array(data7.iloc[::2, :])
data7 = np.array([float(i[0]) for i in data7])

data8 = pd.read_csv('../database/pseudo_observations/AUD.txt')
data8 = np.array(data8.iloc[::2, :])
data8 = np.array([float(i[0]) for i in data8])

data9 = pd.read_csv('../database/pseudo_observations/EUR.txt')
data9 = np.array(data9.iloc[::2, :])
data9 = np.array([float(i[0]) for i in data9])

data10 = pd.read_csv('../database/pseudo_observations/IEAC.txt')
data10 = np.array(data10.iloc[::2, :])
data10 = np.array([float(i[0]) for i in data10])

In [17]:
import pyvinecopulib as pv
controls = pv.FitControlsVinecop(family_set=[pv.BicopFamily.gaussian,
                                             pv.BicopFamily.clayton,
                                             pv.BicopFamily.frank,
                                             pv.BicopFamily.gumbel,
                                             pv.BicopFamily.bb1])

u = np.stack([data1, data2, data3, data4, data5, data6, data7, data8, data9, data10]).T
cop = pv.Vinecop(u, controls=controls)

u_sim = cop.simulate(100)
np.savetxt("u_sim.txt", u_sim, delimiter=",")
print(u_sim)

[[0.79772463 0.58150969 0.61085637 0.81521578 0.63448929 0.25333872
  0.82333231 0.57785713 0.42153504 0.5964131 ]
 [0.77390809 0.58765001 0.53034168 0.54839427 0.60758174 0.29195238
  0.58243041 0.3839604  0.46580821 0.54777563]
 [0.26670714 0.32536818 0.32936413 0.18667356 0.19083258 0.78696727
  0.23736657 0.48932501 0.67931916 0.28359091]
 [0.05779676 0.20909012 0.21455327 0.09016981 0.11556966 0.90683921
  0.18668199 0.20524079 0.93598611 0.12675818]
 [0.62532737 0.75088301 0.74290244 0.64454536 0.69767896 0.31432618
  0.75523219 0.84805783 0.29995938 0.71363633]
 [0.34895608 0.58809409 0.33367855 0.04082125 0.45088984 0.9305669
  0.34509566 0.06232354 0.8507131  0.40126902]
 [0.19985629 0.12326439 0.10256493 0.19369392 0.05962854 0.41448531
  0.74914276 0.10366589 0.88784204 0.07960795]
 [0.74774783 0.86463727 0.74019025 0.79043543 0.76655704 0.14503619
  0.77383274 0.29580822 0.2459816  0.80576824]
 [0.51805768 0.60123982 0.65529561 0.76212286 0.7966484  0.26915625
  0.85968729 

In [18]:
cop.structure

<pyvinecopulib.RVineStructure>
6 4 4 7 5 5 3 2 10 10 
4 7 7 5 1 3 2 10 2 
7 5 5 1 3 2 10 3 
5 8 1 3 2 10 5 
8 1 3 2 10 1 
1 3 2 10 7 
3 2 10 4 
2 10 8 
10 6 
9 

In [19]:
from database.database_parser import create_connection
from database.database_parser import select_curr_solution

test_connection = create_connection('../database/abacus_database.db')

In [20]:
from database.database_parser import select_price_data

def generate_volatility(params, data):
    n_observations = len(data)
    initial_squared_vol_estimate = (params[0]
                                        + params[1] * (data[0] ** 2)
                                        + params[3] * (data[0] ** 2) * np.where(data[0] < 0, 1, 0)
                                        + params[2] * (data[0] ** 2))
    current_squared_vol_estimate = initial_squared_vol_estimate

    for i in range(0, n_observations):
        current_squared_vol_estimate = (params[0] + params[1] * (data[i - 1] ** 2)
                                            + params[3] * (data[i - 1] ** 2) * np.where(data[i - 1] < 0, 1, 0)
                                            + params[2] * current_squared_vol_estimate)

    return np.sqrt(current_squared_vol_estimate)

def get_volatility(asset: str):
    opt_sol = select_curr_solution(test_connection, asset)
    adj_close = select_price_data(test_connection, '1321')
    log_returns = np.array(np.log(adj_close / adj_close.shift(1))[1:])
    return generate_volatility(params=opt_sol, data=log_returns)

In [7]:
from distributions.student_poisson_mixture import spm
# Calculate all inverse samples.
def generate_samples(us, asset):
    opt_sol = select_curr_solution(test_connection, asset)
    mu = opt_sol[4]
    sigma = get_volatility(asset)[0]
    kappa = opt_sol[5]
    lamb = opt_sol[6]
    nu = opt_sol[7]

    for p in us:
        s = spm.ppf(p, mu, sigma, kappa, lamb, nu)
        print(s.root)

generate_samples(u_sim[:, 4], 'IVV')

1.0584664836470876
0.29376358048548606
-0.018547013236331327
-0.018540169082710785
0.9373604137841106
-0.009370369113802086
-0.11197432799522992
-0.014153200520035035
0.01968628690954487
0.048521438908058705
-0.034856268800176095
-0.018702032196939347
0.009192375284793185
-0.04457269369490421
0.5549821022052526
-0.030350200842128368
0.3424838732209676
0.04803011991686046
-0.07242956460401717
0.029592015512437717
0.061801177424709794
0.12858739572443773
-0.04321951457804534
-0.042140300131046894
0.07754664967959247
-0.044797028327946076
0.09282195310411946
-0.019354058068346197
0.07639690173581754
-0.06956005699463562
-0.019735101098384895
0.04194976724074194
0.09362345430676913
-0.04692863344541148
-0.025877356333441622
0.07485770663120202
-0.5855520035975932
0.8069473200891641
0.013652933819617517
-0.04390634923081404
0.014466340724592618
-0.009486637105158043
-0.006765109687165978
-0.013501740080299217
0.18576803323188584
-0.04282738737880735
0.05019180971988053
0.06142885116119932
0

ValueError: f(a) and f(b) must have different signs

In [10]:
df_ret = pd.read_csv('../database/return_samples/IVV.txt')
len(df_reta)

76